<a href="https://colab.research.google.com/github/ASO1801038/AI_teach2020/blob/master/2020AI0404_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Doc2Cevで文章を学習したコーパスモデルを作って保存

### データリストの作成

In [42]:

#学習対象とする青空文庫の作品リスト --- (*1)
list = [
    {"author":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"author":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"author":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"},
    ]},
    {"author":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"},
    ]},
]

### MeCabのインストール

In [43]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab-python3 > /dev/null

# MeCabの実行時の指定パスをインストールパスにリンクさせる
# シンボリックリンク（/etc/mecabrcを/usr/local/etc/mecabrcで参照できるようにする）
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


###データリストを検索して順番に返す関数：book_listを定義

In [44]:
#作品リストをループして順次リターン
def book_list():
  for novellist in list:
    author = novellist["author"]
    for book in novellist["books"]:
      #yieldで順次、値をリターン、次ループへ
      yield author, book

###指定された書籍名（book）のzipファイルを開き、テキストデータを読み込んでリターン

In [45]:
import zipfile
import os.path
import urllib.request as req
#zipファイルを開いて中の文書データを取得
def read_book(author, book):
  zipname = book["zipname"]
  #zipがなければDL
  if not os.path.exists(zipname):
    req.urlretrieve(author["url"] + zipname, zipname)
  zipname = book["zipname"]
  #zipを開く
  with zipfile.ZipFile(zipname, "r") as zf:
    #zipに含まれるファイルを開く
    for filename in zf.namelist():
      #.txtのみ処理する
      if os.path.splitext(filename)[1] != ".txt":
        continue
      with zf.open(filename, "r") as f:
        #文字コードを指定（Shift-JIS）して解読
        return f.read().decode("shift-jis")

###MeCabインスタンス生成と、データと配列にしてリターン

In [46]:
import MeCab
mecab = MeCab.Tagger()

def split_words(text):
  node = mecab.parseToNode(text)
  wakati_words = []
  while node is not None:
    hinshi = node.feature.split(",")[0]
    if hinshi is ["名詞"]:
      wakati_words.append(node.surface)
    elif hinshi in ["名詞","形容詞"]:
      wakati_words.append(node.feature.split(",")[6])
    node = node.next
  return wakati_words

###学習モデル作成メイン処理（zipを開いて学習させる）

In [47]:
from gensim import models
from gensim.models.doc2vec import TaggedDocument

documents = []
for author, book in book_list():
  words = read_book(author, book)
  wakati_words = split_words(words)
  document = TaggedDocument(wakati_words, [author["name"] + ":" +book["name"]])
  print(document)
  documents.append(document)

model = models.Doc2Vec(documents, dm=0, vector_size=300, window=15, min_count=1)
model.save("aozora.model")
print("モデル作成完了")



TaggedDocument(['銀河', '鉄道', '夜', '宮沢', '賢治', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'テキスト', '中', '記号', 'ルビ', '例', '言', 'い', 'われ', 'ルビ', '文字', '列', '始まり', '特定', '記号', '例', '一', '袋', 'ふく', 'ろ', '*', '入力', '者', '注', '主', '外字', '説明', '傍点', '位置', '指定', '数字', '*', '*', '*', '面', '区点', '番号', '底本', 'ページ', '行', '数', '例', '*', '小書き', '平仮名', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '５', '字', '下げ', '一', '午後', '授業', '*', '一', '午後', '授業', '見出し', 'みなさん', 'ふう', '川', '言', 'い', 'われ', '乳', 'あと', '言', 'い', 'われ', 'ぼんやり', '白い', 'もの', 'ほんとう', '何', '承知', 'しょ', 'うち', '先生', '黒板', 'こい', 'ばん', '黒い', '星座', '図', '上', '下', '白い', '銀河', '帯', 'ぎん', 'たい', 'よう', 'ところ', '指', 'みんな', '問', 'い', '*', '手', 'それ', '四', '五', '人', '手', '*', '手', '急', 'いそ', 'い', 'あれ', 'みんな', '星', 'いつか',

###コーパスモデルを使って、関連度の高い文章を推論

In [48]:
#ファイルを読み込み
from gensim import models
model = models.Doc2Vec.load("aozora.model")

###指定されたzipをDLし、テキストデータを読み込んでリターン

In [49]:
import zipfile
import os.path
import urllib.request as req

def read_book(url, zipname):
  if not os.path.exists(zipname):
    req.urlretrieve(url, zipname)
  with zipfile.ZipFile(zipname, "r") as zf:
    for filename in zf.namelist():
      with zf.open(filename, "r") as f:
        #文字コードを指定（Shift-JIS）して解読
        return f.read().decode("shift-jis")

In [53]:
def similar(title, url):
  zipname = url.split("/")[-1]

  words = read_book(url, zipname)
  wakati_words = split_words(words)
  vector = model.infer_vector(wakati_words)
  print("---[",title,"]と似た作品は？---")
  #結果表示
  print(model.docvecs.most_similar([vector], topn=3))
  print("")

###推論メイン処理

In [54]:
similar("宮沢 賢治:よだかの星","https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛","https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:ロジェエの怪事件","https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:末摘花","https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

---[ 宮沢 賢治:よだかの星 ]と似た作品は？---
[('宮澤 賢治:やまなし', 0.9875593185424805), ('宮澤 賢治:セロ弾きのゴーシュ', 0.9823317527770996), ('宮澤 賢治:どんぐりと山猫', 0.9817293882369995)]

---[ 芥川 龍之介:犬と笛 ]と似た作品は？---
[('芥川 竜之介:老年', 0.9706053733825684), ('芥川 竜之介:鼻', 0.9697204828262329), ('ポー エドガー・アラン:沈黙', 0.9666712284088135)]

---[ ポー エドガー・アラン:ロジェエの怪事件 ]と似た作品は？---
[('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.9489631652832031), ('ポー エドガー・アラン:黒猫', 0.8869026899337769), ('ポー エドガー・アラン:落穴と振子', 0.8445948958396912)]

---[ 紫式部:末摘花 ]と似た作品は？---
[('紫式部:源氏物語 04 夕顔', 0.9953033924102783), ('紫式部:源氏物語 02 帚木', 0.9938474297523499), ('紫式部:源氏物語 05 若紫', 0.9916704297065735)]

